In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=1ffb65cab9354089adf1d292633804bf373d7ff6e6d6389cebe4b08790aa0e00
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import re
import rouge_score
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from transformers import DataCollatorForSeq2Seq
from evaluate import load

## dataset preprocess

In [ ]:
dataset = load_dataset("bilalRahib/fiqa-personal-finance-dataset")
train_dataset = dataset["train"]
df1 = train_dataset.to_pandas()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


data/train-00000-of-00001.parquet:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1987 [00:00<?, ? examples/s]

In [ ]:
df1.head()

,input,output
0,Does U.S. tax code call for small business own...,I am going to keep things very simple and expl...
1,What are 'business fundamentals'?,From http://financial-dictionary.thefreedictio...
2,Business Investment Loss from prior year,You need to give specific dates! In the United...
3,Would the purchase of a car for a business thr...,You don't say what country you live in. If it'...
4,Why is “cheque cashing” a legitimate business?,"In my experience (in the US), the main draw of..."


In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp 5th sem/finance tutor/final.csv')
df2.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df2.head()

,input,output
0,"Thanks. I'm pleased with the recent gains, but...",That's a valid concern. While the market's up ...
1,Should I be worried about recent market swings...,Current rate at 1.39% supports continued growt...
2,How does my moderate risk profile fit into thi...,"Your portfolio balances growth and stability, ..."
3,Are we positioned to capitalize on AI and rene...,"Absolutely, we’ve increased allocations in AI-..."
4,What about potential corrections—should I be w...,"While corrections are normal, our strategy emp..."


In [ ]:
df = pd.concat([df1, df2])
df.head()

,input,output
0,Does U.S. tax code call for small business own...,I am going to keep things very simple and expl...
1,What are 'business fundamentals'?,From http://financial-dictionary.thefreedictio...
2,Business Investment Loss from prior year,You need to give specific dates! In the United...
3,Would the purchase of a car for a business thr...,You don't say what country you live in. If it'...
4,Why is “cheque cashing” a legitimate business?,"In my experience (in the US), the main draw of..."


In [ ]:
df.shape

(18085, 2)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18085 entries, 0 to 16097
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   input   18085 non-null  object
 1   output  18051 non-null  object
dtypes: object(2)
memory usage: 423.9+ KB


In [ ]:
df.dropna(axis=0, inplace= True)

In [ ]:
df.shape

(18051, 2)

In [ ]:
def clean(text):
    text = text.strip()
    text = text.replace("\n", " ")
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[^a-zA-Z0-9.,?;:!%$€₹\- ]", "", text)  # keep basic punctuation
    text = text.lower()
    return text

df["input"] = df["input"].apply(clean)
df["output"] = df["output"].apply(clean)

df.to_csv("/content/drive/MyDrive/Colab Notebooks/nlp 5th sem/finance tutor/final_combined.csv", index=False)

In [ ]:
df.head()

,input,output
0,does u.s. tax code call for small business own...,i am going to keep things very simple and expl...
1,what are business fundamentals?,from http:financial-dictionary.thefreedictiona...
2,business investment loss from prior year,you need to give specific dates! in the united...
3,would the purchase of a car for a business thr...,you dont say what country you live in. if its ...
4,why is cheque cashing a legitimate business?,"in my experience in the us, the main draw of c..."


## model

In [ ]:
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(examples):
    model_inputs = tokenizer(
        examples["input"],
        max_length=256,
        truncation=True,
        padding="max_length"
    )

    labels = tokenizer(
        examples["output"],
        max_length=256,
        truncation=True,
        padding="max_length"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess, batched=True)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/1987 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./finchatbot",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2
)

dataset_split = tokenized_dataset["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

trainer.train()

In [ ]:
training_args = TrainingArguments(
    output_dir="./finchatbot",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=30,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2
)

dataset_split = tokenized_dataset["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipython-input-12654736.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jigyasa260604 (jigyasa260604-bennett-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,3.943800,3.128609
2,3.068000,2.787807
3,2.998900,2.765051
4,2.972200,2.755373
5,2.879700,2.748524
6,2.872800,2.744653
7,2.945100,2.741712
8,2.905900,2.739280
9,2.875300,2.737191
10,2.813300,2.736654


TrainOutput(global_step=5970, training_loss=2.885758703158329, metrics={'train_runtime': 2348.3208, 'train_samples_per_second': 20.3, 'train_steps_per_second': 2.542, 'total_flos': 4430700179619840.0, 'train_loss': 2.885758703158329, 'epoch': 30.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/nlp 5th sem/finance tutor/finchatbot-final")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/nlp 5th sem/finance tutor/finchatbot-final")

('/content/drive/MyDrive/Colab Notebooks/nlp 5th sem/finance tutor/finchatbot-final/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/nlp 5th sem/finance tutor/finchatbot-final/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/nlp 5th sem/finance tutor/finchatbot-final/spiece.model',
 '/content/drive/MyDrive/Colab Notebooks/nlp 5th sem/finance tutor/finchatbot-final/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/nlp 5th sem/finance tutor/finchatbot-final/tokenizer.json')

In [ ]:
chatbot = pipeline("text2text-generation", model="/content/drive/MyDrive/Colab Notebooks/nlp 5th sem/finance tutor/finchatbot-final", tokenizer=tokenizer)

def chatbot_qa(ques):
  response = chatbot(ques, max_length=100, do_sample=True, top_p=0.9, temperature=0.7, early_stopping=True,repetition_penalty=2.5)
  return response[0]['generated_text']

user_query = "What is mutual fund?"
answer = chatbot_qa(user_query)
print("Chatbot answer:", answer)

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Chatbot answer: Mutual fund is a mutual fund that invests in stocks, bonds, and other securities. It's not a financial institution, it's an investment company. A mutual fund is a type of institutional fund that invests in the stock market. If you are looking for a mutual fund to invest in, then you might want to consider investing in a mutual fund. You can invest in a mutual fund at any time by investing in a mutual fund. For example, if you invest in a bond fund, you may be able to invest in a mutual fund with a higher interest rate than a mutual fund. However, if you don't invest in a mutual fund, you will probably need to invest in a mutual fund instead of a mutual fund.


In [ ]:
rouge = evaluate.load("rouge")

preds, refs = [], []

for i in range(50):
    input_text = eval_dataset[i]["input"]
    ref_text = eval_dataset[i]["output"]

    pred = chatbot(input_text, max_length=500, do_sample=False)[0]['generated_text']

    preds.append(pred)
    refs.append(ref_text)

results = rouge.compute(predictions=preds, references=refs)
print(results)

Both `max_new_tokens` (=256) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


{'rouge1': np.float64(0.12039984495512293), 'rouge2': np.float64(0.012021518516038845), 'rougeL': np.float64(0.10191243611283335), 'rougeLsum': np.float64(0.10184289819482323)}
